In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_FDN_input_values, read_and_check_USB_input_values
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import optimize_with_correction
from functions import solve_linear_program
from functions import  remove_old_recipes
from functions import  gestion_resultats
from functions import  gestion_FDNresultats

In [4]:
def create_optimal_recipe(recette,table_mp, mp_constraints, elmt_quality_constraints,dossier):

    df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]
    df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)

    # Suppression des matières premières indisponibles dans
    table_mp = clean_table_mp(table_mp, df_mp_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)
    
    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)


    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'

    # Résolution de la formulation 1
    coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    coefficients = [0.85, 0.9, 0.95,1]
    res1, erreurs1 = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)
    
    # Résolution de la formulation 2
    res2, erreurs2 = solve_linear_program(C, constraints, bounds,method)
    
    # # Gestion du resultats USB
    # gestion_resultats(erreurs1, res1, erreurs2, res2, 
    #                   df_mp_dispo, table_mp, constraints, dossier, recette)
    
    # Gestion du resultats pour FDN
    gestion_FDNresultats(erreurs1, res1, erreurs2, res2, df_mp_dispo, table_mp, constraints, dossier, recette)

    return 

In [55]:
dossier_InputsOutputs = os.path.join('.', 'data', 'InputsOutput')
# Suppression du vieux resultats
remove_old_recipes(dossier_InputsOutputs)

# Vérifications des données d'entrée
recette = 'GS' # Nom de la recette
table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_FDN_input_values(dossier_InputsOutputs,recette)



df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]
df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)

# Suppression des matières premières indisponibles dans
table_mp = clean_table_mp(table_mp, df_mp_indispo)

# Construction de la matrice A et du vecteur C
A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)

# Initialisation des listes pour les contraintes
constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

# Mettre  les contraintes concernant les éléments
constraints = format_constraints_elements(df_contraints_element, A,constraints)

# Mettre  les contraintes concernant la qualité
constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)


import numpy as np



def Transpose_dataframe(df):
    """
    Transpose le DataFrame, définissant la première colonne comme noms de colonnes,
    puis déplace les index dans une nouvelle colonne, et enfin supprime le nom de l'index.
    
    Args:
    - df: DataFrame à transposer
    
    Returns:
    - DataFrame Transposé
    """
    # Transposer le DataFrame et définir la première colonne comme noms de colonnes
    df_transposed = df.set_index(df.columns[0]).transpose()
    
    # Déplacer les index dans une nouvelle colonne
    df_transposed.reset_index(inplace=True)
    df_transposed = df_transposed.rename(columns={'index': df.columns[0]})
    
    # Supprimer le nom de l'index
    df_transposed = df_transposed.rename_axis(None, axis=1)
    
    return df_transposed


df_contraints_element = Transpose_dataframe(df_contraints_element)



A.shape[0], df_contraints_element.shape

n = A.shape[0]
for index, row in df_contraints_element.iterrows():
    if not pd.isna(row['Valeur visée']):
        composant = row['Composant'] + '_visee'
        E = np.zeros(n)
        E[index] = 1
        E_dot_A  = E@A
        constraints['A_eq'][composant] = A[index]
        constraints['b_eq'][composant] = pd.to_numeric(row['Valeur visée'], errors='coerce')


C_visee
[1.00e-01 3.50e+00 0.00e+00 4.25e+00 9.80e+01 1.00e-01 1.00e-01 0.00e+00
 6.75e+00 1.55e+00 2.00e-02 0.00e+00 1.00e+02 1.00e-01]
Si_visee
[1.600e-02 1.900e+00 0.000e+00 7.630e-01 0.000e+00 3.000e+00 2.000e-02
 7.500e+01 4.300e-01 9.544e+01 6.500e-01 0.000e+00 0.000e+00 3.050e-01]
Mn_visee
[3.500e-01 4.500e-01 0.000e+00 3.800e-02 0.000e+00 1.500e-01 4.500e-01
 0.000e+00 7.533e+01 7.533e+01 1.500e+00 0.000e+00 0.000e+00 9.800e-01]


In [57]:
A

array([[1.000e-01, 3.500e+00, 0.000e+00, 4.250e+00, 9.800e+01, 1.000e-01,
        1.000e-01, 0.000e+00, 6.750e+00, 1.550e+00, 2.000e-02, 0.000e+00,
        1.000e+02, 1.000e-01],
       [1.600e-02, 1.900e+00, 0.000e+00, 7.630e-01, 0.000e+00, 3.000e+00,
        2.000e-02, 7.500e+01, 4.300e-01, 9.544e+01, 6.500e-01, 0.000e+00,
        0.000e+00, 3.050e-01],
       [3.500e-01, 4.500e-01, 0.000e+00, 3.800e-02, 0.000e+00, 1.500e-01,
        4.500e-01, 0.000e+00, 7.533e+01, 7.533e+01, 1.500e+00, 0.000e+00,
        0.000e+00, 9.800e-01],
       [2.000e-02, 2.000e-01, 1.000e+02, 1.000e-03, 0.000e+00, 1.250e-01,
        1.000e-02, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 4.100e-02],
       [3.200e-02, 1.600e-01, 0.000e+00, 1.900e-02, 0.000e+00, 3.500e-02,
        4.000e-02, 0.000e+00, 0.000e+00, 0.000e+00, 1.400e-01, 0.000e+00,
        0.000e+00, 5.100e-02],
       [7.000e-03, 4.000e-02, 0.000e+00, 2.700e-02, 0.000e+00, 1.000e-02,
        1.000e-02, 0.000e+00, 1

##### FDN

In [5]:
if __name__ == "__main__":
    dossier_InputsOutputs = os.path.join('.', 'data', 'InputsOutput')
    # Suppression du vieux resultats
    remove_old_recipes(dossier_InputsOutputs)

    # Vérifications des données d'entrée
    recette = 'GS' # Nom de la recette
    table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_FDN_input_values(dossier_InputsOutputs,recette)

    # Resolutions du nouveau probleme 1 et 2
    if not errors :
        create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_InputsOutputs)

        

Le problème pour la recette GS admet une solution uniquement pour la deuxième formulation.


##### USB

In [3]:
if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    # Suppression du vieux resultats
    remove_old_recipes(dossier_data)

    # Vérifications des données d'entrée
    Recettes = [col for col in pd.read_excel(chemin_fichier, engine='openpyxl', sheet_name=2).columns if 'Unnamed' not in col] # 'calamine'
    table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_USB_input_values(chemin_fichier)

    # Resolutions du nouveau probleme
    if not errors :
        for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
            create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)



Processing recipes:  40%|████      | 2/5 [00:00<00:00,  6.34recipe/s]

Le problème pour la recette GS 400-15 admet une solution pour les deux formulations.
Le problème pour la recette GS 450-10 admet une solution pour les deux formulations.


Processing recipes:  80%|████████  | 4/5 [00:00<00:00,  6.10recipe/s]

Le problème pour la recette GS 500-7 admet une solution pour les deux formulations.
Le problème pour la recette GS 600-3 admet une solution pour les deux formulations.


Processing recipes: 100%|██████████| 5/5 [00:00<00:00,  5.55recipe/s]

Le problème pour la recette GL 250 admet une solution pour les deux formulations.
